In [ ]:
import pandas as pd
import numpy as np
import xarray as xr

In [ ]:
dv = pd.read_csv('Direccion_del_viento(06-07-025).csv')
vv = pd.read_csv('Velocidad_del_Viento(06-07-025).csv')
hu = pd.read_csv('Humedad_del_Aire(06-07-025).csv')
pr = pd.read_csv('Precipitacion(06-07-025).csv')
pa = pd.read_csv('Presion_atmosferica(06-07-025).csv')
tp = pd.read_csv('Temperatura_Ambiente_del_Aire(06-07-025).csv')

In [41]:
def trim(df):
    df = df.drop(columns=['CodigoSensor', 'NombreEstacion', 'Departamento', 'Municipio', 'ZonaHidrografica', 'Latitud', 'Longitud', 'DescripcionSensor', 'UnidadMedida'])
    df.drop_duplicates()
    df = df[df["CodigoEstacion"].isin([21205012,21205710,21205791,21206600,21206940,21206960])]
    df["ValorObservado"] = pd.to_numeric(df["ValorObservado"], errors="coerce")
    df['FechaObservacion'] = pd.to_datetime(df['FechaObservacion'], format="%m/%d/%Y %I:%M:%S %p")
    df = df.loc[df['FechaObservacion'] >= '2010-01-01']
    df['Fecha'] = df['FechaObservacion'].dt.date
    df['Hora'] = df['FechaObservacion'].dt.hour
    df = df.drop(columns=['FechaObservacion'])
    return df
        


In [42]:
def reshape(df):
    dates    = pd.date_range(df["Fecha"].min(), df["Fecha"].max(), freq="D")
    hours    = np.arange(24)
    stations = df["CodigoEstacion"].unique()
    full_idx = pd.MultiIndex.from_product(
        [dates, hours, stations],
        names=["Fecha","Hora","CodigoEstacion"]
    )
    s = ( df
      .set_index(["Fecha","Hora","CodigoEstacion"])["ValorObservado"]
      .reindex(full_idx)
    )
    return s

In [43]:
def interpolate(df):
    df = df.interpolate_na(dim="CodigoEstacion", method="nearest", fill_value="extrapolate")
    df = df.interpolate_na(dim="Hora", method="nearest", fill_value="extrapolate")
    df = df.interpolate_na(dim="Fecha", method="nearest", fill_value="extrapolate")
    return df

In [44]:
def average_angle(angles):

    angles = np.radians(angles)
    x = np.sum(np.cos(angles))
    y = np.sum(np.sin(angles))
    avg = np.arctan2(y,x)
    
    return avg

In [45]:
def splitXY(array):
    X = np.zeros((array.shape[0]-14,14,array.shape[1],array.shape[2]))
    Y = np.zeros((array.shape[0]-14,array.shape[1],array.shape[2]))
    for day in range(array.shape[0]-14):
        X[day,:,:,:] = array[day:day+14,:,:]
    Y[:,:,:] = array[14:,:,:]
    return X, Y

In [ ]:
pa_trim = trim(pa)
pa_trim = pa_trim.groupby(['Fecha', 'Hora', 'CodigoEstacion']).mean('ValorObservado').reset_index()
pa_shape = reshape(pa_trim)
pa_array = pa_shape.to_xarray()
pa_array = interpolate(pa_array)
pa_X, pa_Y = splitXY(pa_array)
np.save('pa_X.npy', pa_X)

/tmp/ipykernel_29134/865302382.py:5: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  X[day,:,:,:] = array[day:day+14,:,:]
/tmp/ipykernel_29134/865302382.py:6: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  Y[:,:,:] = array[14:,:,:]


In [ ]:
hu_trim = trim(hu)
hu_trim = hu_trim.groupby(['Fecha', 'Hora', 'CodigoEstacion']).apply(average_angle).reset_index()
hu_shape = reshape(hu_trim)
hu_array = hu_shape.to_xarray()
hu_array = interpolate(hu_array)
hu_X, hu_Y = splitXY(hu_array)
np.save('hu_X.npy', hu_X)

/home/kanvolu/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/tmp/ipykernel_29134/865302382.py:5: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  X[day,:,:,:] = array[day:day+14,:,:]
/tmp/ipykernel_29134/865302382.py:6: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  Y[:,:,:] = array[14:,:,:]


In [ ]:
pr_trim = trim(pr)
pr_trim = pr_trim.groupby(['Fecha', 'Hora', 'CodigoEstacion']).mean('ValorObservado').reset_index()
pr_shape = reshape(pr_trim)
pr_array = pr_shape.to_xarray()
pr_array = interpolate(pr_array)
pr_X, pr_Y = splitXY(pr_array)
np.save('pr_X.npy', pr_X)

/tmp/ipykernel_29134/865302382.py:5: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  X[day,:,:,:] = array[day:day+14,:,:]
/tmp/ipykernel_29134/865302382.py:6: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  Y[:,:,:] = array[14:,:,:]


In [ ]:
tp_trim = trim(tp)
tp_trim = tp_trim.groupby(['Fecha', 'Hora', 'CodigoEstacion']).mean('ValorObservado').reset_index()
tp_shape = reshape(tp_trim)
tp_array = tp_shape.to_xarray()
tp_array = interpolate(tp_array)
tp_X, tp_Y = splitXY(tp_array)
np.save('tp_X.npy', tp_X)

/tmp/ipykernel_29134/865302382.py:5: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  X[day,:,:,:] = array[day:day+14,:,:]
/tmp/ipykernel_29134/865302382.py:6: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  Y[:,:,:] = array[14:,:,:]


In [ ]:
dv_trim = trim(dv)
dv_trim = dv_trim.groupby(['Fecha', 'Hora', 'CodigoEstacion']).apply(average_angle).reset_index()
dv_shape = reshape(dv_trim)
dv_array = dv_shape.to_xarray()
dv_array = interpolate(dv_array)

vv_trim = trim(vv)
vv_trim = vv_trim.groupby(['Fecha', 'Hora', 'CodigoEstacion']).mean('ValorObservado').reset_index()
vv_shape = reshape(vv_trim)
vv_array = vv_shape.to_xarray()
vv_array = interpolate(vv_array)

vx = np.zeros(vv_array.shape)
vy = vx
vx = np.multiply(vv_array, np.cos(dv_array))
vy = np.multiply(vv_array, np.sin(dv_array))


vx_X, vx_Y = splitXY(vx)
np.save('vx_X.npy', vx_X)


vy_X, vy_Y = splitXY(vy)
np.save('vy_X.npy', vy_X)

/tmp/ipykernel_29134/865302382.py:5: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  X[day,:,:,:] = array[day:day+14,:,:]
/tmp/ipykernel_29134/865302382.py:6: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  Y[:,:,:] = array[14:,:,:]


In [80]:
Y = np.zeros((4988,24,6,6))
metrics = [hu_Y, pa_Y, vx_Y, vy_Y, tp_Y, pr_Y]
for metric in range(len(metrics)):
    Y[:,:,:,metric] = metrics[metric]
np.save('Y.npy', Y)